In [17]:
#Import packages
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
lm = LinearRegression()

import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

#Load our two Data Frames
AmesDummies = pd.read_csv('AmesDummies.csv')
AmesDummiesOrdinal = pd.read_csv('AmesDummiesOrdinal.csv')

In [18]:
np.random.seed(19)
testIdxes = np.random.choice(range(1458), size= 292, replace=False)
trainIdxes = list(set(range(1458))-set(testIdxes))

## First, let's remove obvious features that will not affect our analysis

In [19]:
#Let's try the most basic linear regression, just to get a sense of what the results look like:
#THIS IS WITH THE WHOLE DATA SET -- WE WILL CUT TO OUR TRAIN SET NEXT
AmesDummiesOrdinalX = AmesDummiesOrdinal.drop('SalePrice', axis=1)
AmesDummiesOrdinalY = AmesDummiesOrdinal['SalePrice']

X = AmesDummiesOrdinalX
Y = AmesDummiesOrdinalY

X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     82.77
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:09:47   Log-Likelihood:                -16648.
No. Observations:                1458   AIC:                         3.367e+04
Df Residuals:                    1272   BIC:                         3.465e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.645e+

In [20]:
#Now, to Feature Select, let's trim our AmesDummiesOrdinal to the Train subset:
AmesDummiesOrdinal = AmesDummiesOrdinal.iloc[trainIdxes,]

In [21]:
#Let's get rid of "Id" (the index from the Processing DF), and any features with 5 or fewer observations
AmesDummiesOrdinal = AmesDummiesOrdinal.drop('Id', axis=1)

AmesDummiesOrdinalX = AmesDummiesOrdinal.drop('SalePrice', axis=1)
AmesDummiesOrdinalY = AmesDummiesOrdinal['SalePrice']

import statsmodels.api as sm
X = AmesDummiesOrdinalX
Y = AmesDummiesOrdinalY

X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     67.72
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:09:47   Log-Likelihood:                -13335.
No. Observations:                1166   AIC:                         2.703e+04
Df Residuals:                     985   BIC:                         2.795e+04
Df Model:                         180                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.816e+

In [22]:
'''Finally, we want to eliminate GarageAge. This is troublesome because houses without a garage have no date, and we can't
dummify this.'''
AmesDummiesOrdinal = AmesDummiesOrdinal.drop(['GarageAge'], axis=1)

AmesDummiesOrdinalX = AmesDummiesOrdinal.drop('SalePrice', axis=1)
AmesDummiesOrdinalY = AmesDummiesOrdinal['SalePrice']

import statsmodels.api as sm
X = AmesDummiesOrdinalX
Y = AmesDummiesOrdinalY

X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     68.15
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:09:48   Log-Likelihood:                -13335.
No. Observations:                1166   AIC:                         2.703e+04
Df Residuals:                     986   BIC:                         2.794e+04
Df Model:                         179                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.781e+

In [23]:
#At this point, we have 158 possible features and one dependent variable ('SalePrice')
AmesDummiesOrdinal.shape

(1166, 185)

## Now, let's look for features with high multicollinearity and address them. We could manually do this by removing features with high VIF one at a time, or could manually search through the correlation of each variable with others first. Let's compare the two methods

In [24]:
'''#This is an imported function, found online, to check VIF for a given DF and remove features with too high a VIF:

def calculate_vif_(X, thresh=100):
    X2 = X.copy()
    cols = X2.columns
    variables = np.arange(X2.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X2[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X2[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc) + " -- VIF: " + str(max(vif)))
            variables = np.delete(variables, maxloc)
            dropped=True

    print('Remaining variables:')
    print(X2.columns[variables])

    return X2[cols[variables]]'''

'#This is an imported function, found online, to check VIF for a given DF and remove features with too high a VIF:\n\ndef calculate_vif_(X, thresh=100):\n    X2 = X.copy()\n    cols = X2.columns\n    variables = np.arange(X2.shape[1])\n    dropped=True\n    while dropped:\n        dropped=False\n        c = X2[cols[variables]].values\n        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]\n\n        maxloc = vif.index(max(vif))\n        if max(vif) > thresh:\n            print(\'dropping \'\' + X2[cols[variables]].columns[maxloc] + \'\' at index: \' + str(maxloc) + " -- VIF: " + str(max(vif)))\n            variables = np.delete(variables, maxloc)\n            dropped=True\n\n    print(\'Remaining variables:\')\n    print(X2.columns[variables])\n\n    return X2[cols[variables]]'

In [25]:
#Create an AmesDummies DF with VIF's under 10, removing one at a time, using the above function:
#AmesDummiesVIFUnder10 = calculate_vif_(AmesDummiesOrdinal.drop('SalePrice', axis=1), thresh=10)

In [26]:
'''This method removed a number of variables that one would think would be important for our model -- age of house, TotalSF,
size of garage, number of rooms/baths, basement and garage quality, etc. It may not know which of two similar features to 
discriminate on, and be choosing more obscure ones in favor of clearer and more descriptive variables. Let's try doing this 
manually.'''

"This method removed a number of variables that one would think would be important for our model -- age of house, TotalSF,\nsize of garage, number of rooms/baths, basement and garage quality, etc. It may not know which of two similar features to \ndiscriminate on, and be choosing more obscure ones in favor of clearer and more descriptive variables. Let's try doing this \nmanually."

In [27]:
#Devise a function to produce a correlation matrix for our feature DF, then go manually
def CreateCorrelationMatrix(df, dependent):
    df2 = df.drop(dependent, axis=1)
    for i in range(len(df2.columns)):
        corrarray = []
        indexarray = []
        for j in range(len(df2.columns)):
            corr12 = df2[df2.columns[i]].corr(df2[df2.columns[j]])
            corrarray.append(corr12)
            indexarray.append(df2.columns[j])
        seriesi = pd.Series(corrarray, index=indexarray)
        
        if i > 0:
            corrDF = pd.concat([corrDF, seriesi], axis=1)
        
        else:
            corrDF = pd.DataFrame(seriesi)
     
    #Rename the columns to be the same as the indices (a self matrix)
    corrDF.columns = corrDF.index
    
    #reset all self-covariances to 0
    for var in corrDF.columns:
        corrDF.loc[var, var] = 0
    
    return corrDF   

In [28]:
#Create the overall correlation matrix, first with all features:
corrDF0 = CreateCorrelationMatrix(AmesDummiesOrdinal, 'SalePrice')

In [29]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['BldgType_Duplex'].sort_values(ascending=False) #Correlation=1 with MSSubClass_90. Keep Duplex, more descriptive

MSSubClass_90            1.000000
KitchenAbvGr             0.728572
SaleCondition_Alloca     0.281314
GarageType_CarPort       0.175943
Foundation_Slab          0.175765
BedroomAbvGr             0.166415
GarageType_No            0.161957
RoofMatl_Roll            0.161839
TotRmsAbvGrd             0.157200
HouseStyle_SFoyer        0.131401
YearsSinceRemodel        0.129747
Foundation_CBlock        0.129497
Neighborhood_Mitchel     0.127168
Exterior_Plywood         0.122621
Condition_Feedr          0.119022
Condition_RRAe           0.112606
CentralAir_N             0.112085
RoofStyle_Shed           0.110735
SaleCondition_AdjLand    0.110735
Heating_Wall             0.110735
SaleCondition_Family     0.096391
MSZoning_RH              0.089284
Exterior_AsphShn         0.087388
Electrical_FuseP         0.087388
SaleType_ConLI           0.087388
SaleType_Oth             0.087388
TotalBath                0.084777
Electrical_FuseF         0.077135
Exterior_Stone           0.073058
Exterior_AsbSh

In [30]:
#Remove MSSubClass_90, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [31]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['SaleType_New'].sort_values(ascending=False) #SaleType_New is essentially SaleCondition_Partial (mentioned in the txt also)

SaleCondition_Partial    0.989498
ExterQual                0.388936
MasVnrType_Stone         0.339733
KitchenQual              0.337839
BsmtQual                 0.332781
OverallQual              0.321748
GarageArea               0.307656
Neighborhood_Somerst     0.299875
GarageCars               0.291227
Neighborhood_NridgHt     0.276861
HeatingQC                0.253291
GarageFinish             0.250428
TotalBsmtSF              0.249676
MSZoning_FV              0.236755
OpenPorchSF              0.183304
TotalBath                0.173205
FireplaceQu              0.170387
TotalSF                  0.164842
BsmtExposure             0.162022
MasVnrArea               0.157186
TotRmsAbvGrd             0.143393
Exterior_CemntBd         0.123079
Neighborhood_StoneBr     0.121809
YearsSinceSale           0.121738
Neighborhood_Blmngtn     0.110859
MSSubClass_60            0.109793
LandContour_HLS          0.107688
GarageType_BuiltIn       0.106680
BsmtCond                 0.105901
MoSold_Autumn 

In [32]:
#Remove SaleCondition_Partial, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [33]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['BldgType_2fmCon'].sort_values(ascending=False) #2FmCon and MS190 virtually identical, remove MS190

MSSubClass_190          0.980151
KitchenAbvGr            0.354263
CentralAir_N            0.244521
YearsAgoBuilt           0.205242
SaleType_ConLD          0.198544
GarageType_No           0.183676
Neighborhood_OldTown    0.179612
Neighborhood_SWISU      0.118196
LandContour_Bnk         0.116710
YearsSinceRemodel       0.115109
MSZoning_RH             0.111937
Electrical_FuseP        0.106989
Exterior_MetalSd        0.102199
Alley_Grvl              0.095491
LotArea                 0.095226
Heating_GasW            0.094602
Street_Grvl             0.090480
Exterior_AsbShng        0.090011
BedroomAbvGr            0.087855
MSZoning_RM             0.086318
Heating_Grav            0.078980
GarageType_2Types       0.078980
Foundation_Stone        0.078980
Condition_Artery        0.068821
Foundation_BrkTil       0.068331
EnclosedPorch           0.067761
LowQualFinSF            0.067575
HouseStyle_1.5Fin       0.066930
TotRmsAbvGrd            0.064862
GarageType_CarPort      0.063455
          

In [34]:
#Remove MSSubClass_190, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [35]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['GarageQual'].sort_values(ascending=False) #Garage Quality and Condition are highly correlated. Remove Cond

GarageCond               0.957230
GarageCars               0.592688
GarageArea               0.576137
GarageFinish             0.499104
PavedDrive               0.347523
OverallQual              0.298417
ExterQual                0.240911
KitchenQual              0.232537
FireplaceQu              0.225983
Fireplaces               0.217671
BsmtQual                 0.202321
TotalBsmtSF              0.184211
TotalSF                  0.178343
TotalBath                0.159376
BsmtCond                 0.156043
MasVnrType_BrkFace       0.155881
MSSubClass_60            0.144019
MasVnrArea               0.142425
HeatingQC                0.135903
BsmtScore                0.130078
WoodDeckSF               0.104355
ExterCond                0.102735
LotShape_IR1             0.101115
GarageType_Detchd        0.093314
MasVnrType_Stone         0.084326
LotArea                  0.084040
TotRmsAbvGrd             0.079690
ScreenPorch              0.078836
LotFrontage              0.076240
Exterior_HdBoa

In [36]:
#Remove GarageCond, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [37]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['MSSubClass_80'].sort_values(ascending=False) #MS80 is basically split-level by the description, remove it

HouseStyle_SLvl          0.945493
BsmtExposure             0.196846
MasVnrType_BrkFace       0.140435
GarageType_BuiltIn       0.134605
Fence                    0.128332
Foundation_CBlock        0.119050
Exterior_HdBoard         0.113243
GarageType_Basment       0.097209
LotConfig_FR2            0.090389
Neighborhood_Mitchel     0.086312
Neighborhood_Veenker     0.085388
Exterior_AsphShn         0.078786
SaleType_CWD             0.078786
Exterior_Plywood         0.072048
BsmtScore                0.071223
Neighborhood_Gilbert     0.070839
Exterior_AsbShng         0.060814
GarageQual               0.059748
PoolQC                   0.058912
PavedDrive               0.058381
PoolArea                 0.057379
RoofMatl_WdShake         0.055881
OverallCond              0.054927
SaleCondition_Abnorml    0.054062
Neighborhood_Timber      0.053099
WoodDeckSF               0.051654
RoofMatl_WdShngl         0.048661
MasVnrArea               0.048444
Fireplaces               0.046637
RoofStyle_Mans

In [38]:
#Remove MSSubClass_80, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [39]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['MSSubClass_50'].sort_values(ascending=False) #MS50 is basically a 1.5 story, can be removed

HouseStyle_1.5Fin       0.936111
YearsAgoBuilt           0.369665
Foundation_BrkTil       0.275328
GarageType_Detchd       0.252321
YearsSinceRemodel       0.229501
Condition_Artery        0.220269
Neighborhood_BrkSide    0.205261
Exterior_Wd Sdng        0.195459
MSZoning_RM             0.170979
Neighborhood_OldTown    0.147527
Neighborhood_SWISU      0.145807
Neighborhood_IDOTRR     0.143122
OverallCond             0.131448
EnclosedPorch           0.129579
Alley_Grvl              0.128749
Exterior_MetalSd        0.119487
RoofStyle_Gambrel       0.116796
Electrical_FuseA        0.104210
LowQualFinSF            0.101187
Foundation_Wood         0.100568
LandContour_Bnk         0.099973
Neighborhood_Edwards    0.097241
Exterior_WdShing        0.091306
MSZoning_C (all)        0.090058
Fence                   0.079191
Heating_GasW            0.078773
Electrical_FuseF        0.078696
Heating_Grav            0.069580
Neighborhood_Crawfor    0.063058
CentralAir_N            0.060935
          

In [40]:
#Remove MSSubClass_50, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [41]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['MSSubClass_45'].sort_values(ascending=False) #MS45 is basically a 1.5 story unfinished, can be removed

HouseStyle_1.5Unf       0.893654
MSZoning_RM             0.142855
Neighborhood_BrkSide    0.141439
Electrical_FuseF        0.134300
Condition_RRAe          0.128036
Neighborhood_IDOTRR     0.122693
YearsAgoBuilt           0.097919
LandContour_Bnk         0.088615
OverallCond             0.086920
Exterior_MetalSd        0.083423
EnclosedPorch           0.083251
Foundation_BrkTil       0.078130
Exterior_Wd Sdng        0.075855
GarageType_Detchd       0.068262
GarageType_No           0.067219
YearsSinceRemodel       0.059110
Exterior_WdShing        0.043258
Condition_Artery        0.042322
Alley_Grvl              0.039689
LandContour_HLS         0.036527
LandSlope_Mod           0.033025
Fence                   0.031610
MoSold_Winter           0.031338
Condition_Feedr         0.022731
Electrical_FuseA        0.022283
CentralAir_N            0.020558
Functional              0.020277
YearsSinceSale          0.018814
Neighborhood_OldTown    0.016295
LotConfig_Corner        0.015764
          

In [42]:
#Remove MSSubClass_45, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [43]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['PoolQC'].sort_values(ascending=False) #We are now in correlation values below 0.9, though many rough guides offer 0.7-0.75 as the target range. For PoolQC and PoolArea, 0.9 is too high, remove PoolQC

PoolArea                 0.865471
Exterior_ImStucc         0.227122
MSSubClass_75            0.152152
Fence                    0.150101
LotFrontage              0.148025
TotalSF                  0.145239
EnclosedPorch            0.139355
LowQualFinSF             0.131141
SaleCondition_Abnorml    0.126722
Exterior_Stucco          0.114928
Neighborhood_NoRidge     0.097979
TotalBath                0.092250
Condition_Artery         0.088702
OverallQual              0.074093
TotalBsmtSF              0.074083
Fireplaces               0.068902
LotConfig_Corner         0.066639
BedroomAbvGr             0.063978
Neighborhood_Mitchel     0.062773
TotRmsAbvGrd             0.060665
HouseStyle_SLvl          0.054707
GarageArea               0.050147
KitchenQual              0.048052
HouseStyle_2Story        0.047995
BsmtScore                0.047957
YearsSinceSale           0.045212
Exterior_Plywood         0.041435
LotArea                  0.041058
FireplaceQu              0.038629
ExterCond     

In [44]:
#Remove PoolQC, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolArea']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [45]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['GarageArea'].sort_values(ascending=False) #Garage cars and area are natrually highly correlated. I think SF is a more precise measure, remove GarageCars

GarageCars              0.888263
GarageQual              0.576137
OverallQual             0.562791
GarageFinish            0.533239
ExterQual               0.506234
KitchenQual             0.503881
TotalBsmtSF             0.480150
TotalSF                 0.463478
TotalBath               0.430527
BsmtQual                0.418236
MasVnrArea              0.352363
FireplaceQu             0.345788
TotRmsAbvGrd            0.321182
HeatingQC               0.313503
Neighborhood_NridgHt    0.307911
SaleType_New            0.307656
PavedDrive              0.294227
MasVnrType_Stone        0.291844
Fireplaces              0.272657
MSSubClass_60           0.256807
BsmtExposure            0.251867
LotFrontage             0.242923
OpenPorchSF             0.229473
WoodDeckSF              0.216793
Neighborhood_Somerst    0.198142
MasVnrType_BrkFace      0.193239
BsmtScore               0.190286
LotArea                 0.171519
Neighborhood_NoRidge    0.167353
BsmtCond                0.154513
          

In [46]:
#Remove GarageCars, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolArea', 'GarageCars']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [47]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['FireplaceQu'].sort_values(ascending=False) 
''''FireplaceQu is related to Number of fireplaces. Reading through a detailed description of why this is, the conclusion is that
FireplaceQu would be the slighly better indicator. Often, the "2nd" fireplace would be a small pre=fab fireplace or Franklin
Stove in the basement. FireplaceQu lists the quality of the best fireplace in the house. We should keep that.'''

'\'FireplaceQu is related to Number of fireplaces. Reading through a detailed description of why this is, the conclusion is that\nFireplaceQu would be the slighly better indicator. Often, the "2nd" fireplace would be a small pre=fab fireplace or Franklin\nStove in the basement. FireplaceQu lists the quality of the best fireplace in the house. We should keep that.'

In [48]:
#Remove Fireplaces, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolQC', 'GarageCars', 'Fireplaces']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [49]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['Neighborhood_Somerst'].sort_values(ascending=False) #Neighborhood_Somerset may be a largely "floating village" residenial neighborhood. The actual neighborhood should contain more value, so eliminate MSZoning_FV

MSZoning_FV             0.856975
Alley_Pave              0.389077
SaleType_New            0.299875
MSSubClass_160          0.250536
ExterQual               0.237842
OverallQual             0.211654
MasVnrType_Stone        0.211409
OpenPorchSF             0.207547
KitchenQual             0.202660
HeatingQC               0.200145
GarageArea              0.198142
BsmtQual                0.190298
HouseStyle_2Story       0.146944
BldgType_TwnhsE         0.141364
GarageFinish            0.130171
TotalBath               0.124909
SaleType_Con            0.115057
Exterior_CemntBd        0.083983
MSSubClass_60           0.082591
BldgType_Twnhs          0.081315
Exterior_MetalSd        0.077728
PavedDrive              0.075070
MasVnrArea              0.073640
GarageQual              0.068982
Functional              0.062120
BsmtCond                0.060799
LotConfig_FR3           0.057864
SaleType_CWD            0.057864
Condition_RRAn          0.055387
MoSold_Winter           0.055372
          

In [50]:
#Remove MSZoning_FV, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolArea', 'GarageCars', 'Fireplaces', 'MSZoning_FV']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [51]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['RoofMatl_Tar&Grv'].sort_values(ascending=False) #These two must be mostly related. "Flat" seems a better descriptor than Tar%Grv, however

RoofStyle_Flat           0.856280
LotFrontage              0.170620
Neighborhood_ClearCr     0.156440
Condition_PosA           0.149579
Exterior_BrkComm         0.149579
LandSlope_Mod            0.146236
LandContour_Low          0.145120
GarageType_CarPort       0.137680
LandSlope_Sev            0.128036
Exterior_Plywood         0.124653
MasVnrType_BrkCmn        0.120004
LotShape_IR2             0.114494
BsmtExposure             0.114008
LotArea                  0.107974
Heating_GasW             0.097484
GarageType_Basment       0.089648
Foundation_Slab          0.080322
LotConfig_CulDSac        0.071573
OpenPorchSF              0.069592
SaleCondition_Abnorml    0.067400
ScreenPorch              0.067374
Electrical_FuseF         0.066927
TotalSF                  0.049078
ExterCond                0.043378
LandContour_HLS          0.041282
LotShape_IR1             0.040880
HouseStyle_SLvl          0.039056
YearsSinceRemodel        0.034240
MoSold_Autumn            0.028751
Exterior_Wd Sd

In [52]:
#Remove RoofMatl_Tar&Grv, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolArea', 'GarageCars', 'Fireplaces', 'MSZoning_FV', 'RoofMatl_Tar&Grv']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [53]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
corrDF0['TotRmsAbvGrd'].sort_values(ascending=False) #Total Rooms should naturally correlate with SF, and SF seems a much better descriptor

TotalSF                 0.825145
BedroomAbvGr            0.685216
TotalBath               0.462167
OverallQual             0.436081
HouseStyle_2Story       0.426350
MSSubClass_60           0.423778
FireplaceQu             0.355767
GarageArea              0.321182
ExterQual               0.295076
KitchenQual             0.292001
TotalBsmtSF             0.289238
GarageType_BuiltIn      0.276436
MasVnrArea              0.267777
LotFrontage             0.265904
KitchenAbvGr            0.250967
GarageFinish            0.241321
OpenPorchSF             0.228272
BsmtQual                0.209888
HouseStyle_2.5Fin       0.208821
LotArea                 0.196778
Neighborhood_NoRidge    0.182421
Neighborhood_NridgHt    0.170725
HeatingQC               0.170700
LowQualFinSF            0.165840
BldgType_Duplex         0.157200
WoodDeckSF              0.157135
MSSubClass_75           0.149620
SaleType_New            0.143393
MSSubClass_70           0.136136
MasVnrType_Stone        0.135975
          

In [54]:
#Remove RoofMatl_Tar&Grv, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolArea', 'GarageCars', 'Fireplaces', 'MSZoning_FV', 'RoofMatl_Tar&Grv', 'TotRmsAbvGrd']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [55]:
#Check the matrix out, eliminate a variable then rerun the process:
np.max(corrDF0).sort_values(ascending=False)
#The next 3 are all MSSubClasses, which we should eliminate

BldgType_TwnhsE         0.779141
MSSubClass_120          0.779141
MSSubClass_85           0.775455
HouseStyle_SFoyer       0.775455
HouseStyle_2Story       0.760696
MSSubClass_60           0.760696
KitchenAbvGr            0.728572
BldgType_Duplex         0.728572
OverallQual             0.722220
ExterQual               0.722220
KitchenQual             0.715134
HouseStyle_2.5Unf       0.685624
MSSubClass_75           0.685624
Neighborhood_NPkVill    0.664943
Exterior_BrkComm        0.664943
LowQualFinSF            0.632350
HouseStyle_2.5Fin       0.632350
BsmtQual                0.630755
BsmtCond                0.629026
TotalSF                 0.614572
MSSubClass_160          0.614457
BldgType_Twnhs          0.614457
TotalBath               0.605381
YearsSinceRemodel       0.595028
YearsAgoBuilt           0.595028
MSSubClass_180          0.584870
Neighborhood_MeadowV    0.584870
TotalBsmtSF             0.581484
GarageQual              0.576137
GarageArea              0.576137
          

In [56]:
#Remove additional MSSubclasses, rerun the analysis, and create a list of removed features
RemovedFeatures = ['MSSubClass_90', 'SaleCondition_Partial', 'MSSubClass_190', 'GarageCond', 'MSSubClass_80', 'MSSubClass_50',
                  'MSSubClass_45', 'PoolArea', 'GarageCars', 'Fireplaces', 'MSZoning_FV', 'RoofMatl_Tar&Grv', 'TotRmsAbvGrd',
                  'MSSubClass_120', 'MSSubClass_85', 'MSSubClass_60']
AmesDummiesMultiReduction = AmesDummiesOrdinal.drop(RemovedFeatures, axis=1)
corrDF0 = CreateCorrelationMatrix(AmesDummiesMultiReduction, 'SalePrice')

In [57]:
#Check the matrix out, eliminate a variable then rerun the process:
print(np.max(corrDF0).sort_values(ascending=False))
'''Exterior, Kitchen, and Overall quality are significantly correlated with each other (~0.7), but there is not enough
information here to decide for sure whether to eliminate any. They should be kept for now and moved to the Feature Selection
part of the analysis'''

KitchenAbvGr            0.728572
BldgType_Duplex         0.728572
OverallQual             0.722220
ExterQual               0.722220
KitchenQual             0.715134
MSSubClass_75           0.685624
HouseStyle_2.5Unf       0.685624
Neighborhood_NPkVill    0.664943
Exterior_BrkComm        0.664943
HouseStyle_2.5Fin       0.632350
LowQualFinSF            0.632350
BsmtQual                0.630755
BsmtCond                0.629026
TotalSF                 0.614572
BldgType_Twnhs          0.614457
MSSubClass_160          0.614457
TotalBath               0.605381
YearsSinceRemodel       0.595028
YearsAgoBuilt           0.595028
MSSubClass_180          0.584870
Neighborhood_MeadowV    0.584870
TotalBsmtSF             0.581484
GarageArea              0.576137
GarageQual              0.576137
MasVnrType_BrkFace      0.574884
MasVnrArea              0.574884
Neighborhood_OldTown    0.564227
MSZoning_RM             0.564227
GarageFinish            0.556061
Foundation_BrkTil       0.550490
          

'Exterior, Kitchen, and Overall quality are significantly correlated with each other (~0.7), but there is not enough\ninformation here to decide for sure whether to eliminate any. They should be kept for now and moved to the Feature Selection\npart of the analysis'

In [58]:
AmesDummiesMultiReduction.shape

(1166, 169)

In [59]:
'''After removing obvious features, our DF went from 187 to 158 feature columns. Applying a correlation analysis, this number
was reduced to 142. These remaining columns can bow be subjected to either Backward or Forward feature selection techniques
to decide which ones contribute meaningfully to our model'''

'After removing obvious features, our DF went from 187 to 158 feature columns. Applying a correlation analysis, this number\nwas reduced to 142. These remaining columns can bow be subjected to either Backward or Forward feature selection techniques\nto decide which ones contribute meaningfully to our model'

In [60]:
#We see that the DF we have created, eliminating >40 sparse and co-linear features, still retains a high R^2 (0.919) and lowered AIC
AmesDummiesOrdinalX = AmesDummiesMultiReduction.drop('SalePrice', axis=1)
AmesDummiesOrdinalY = AmesDummiesOrdinal['SalePrice']

X = AmesDummiesOrdinalX
Y = AmesDummiesOrdinalY

X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     73.74
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:13:31   Log-Likelihood:                -13348.
No. Observations:                1166   AIC:                         2.703e+04
Df Residuals:                    1001   BIC:                         2.786e+04
Df Model:                         164                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.639e+

## Now, we need to select features to use in our analysis. Let's first try a method of Forward selection, where we add features that add the most to AIC, until we have no more features that will add value.

In [61]:
#Define method to add features one at a time based on which subtract the most from AIC:
def AddFeatureListbyAIC(df, dependent):
    AnyPositive = True
    df2 = df.copy()
    df2X = df2.drop(dependent, axis=1)
    df2Y = df2[dependent]
    ListofPossibleFeatures = list(df2X.columns)
    StartingFeatureList = []
    CreatedFeatureList = []
    AICEvolutionList = []
    
    while AnyPositive == True:
        ListOfTriedValues = []
        if len(CreatedFeatureList) > 0:
            X2 = sm.add_constant(df2X[CreatedFeatureList])
            est = sm.OLS(df2Y, X2)
            est2 = est.fit()
            AICBase = est2.aic
            AICList = []
        else:
            AICBase = 1000000
            AICList = []
        
        for i in ListofPossibleFeatures:
            if i in CreatedFeatureList:
                continue
            tempDFX = df2X[CreatedFeatureList]
            tempDFX = pd.concat([tempDFX, df2X[[i]]], axis=1)
            tempX2 = sm.add_constant(tempDFX)
            est = sm.OLS(df2Y, tempX2)
            est2 = est.fit()
            AICList.append(est2.aic)
            AICListN = np.array(AICList)
            ListOfTriedValues.append(i)
            
        if any(AICListN-AICBase < 0) == False:
            break
            
        else:
            index = AICList.index(min(AICList))
            AddedValue = ListOfTriedValues[index]
        CreatedFeatureList.append(AddedValue)
        
        AICEvolutionList.append(AICList[index])
        #df2X = df2X.drop(AddedValue, axis=1)
        StartingFeatureList = list(df2X.columns)
    
    resultDF = pd.DataFrame({'CreatedFeatures': np.array(CreatedFeatureList), 'NewFScore': np.array(AICEvolutionList)})
        
    return resultDF

In [62]:
AmesDummiesForwardAICList = AddFeatureListbyAIC(AmesDummiesMultiReduction, 'SalePrice')

In [63]:
AmesDummiesForwardAICList

,CreatedFeatures,NewFScore
0,OverallQual,28528.961392
1,TotalSF,28096.612332
2,TotalBsmtSF,27852.698181
3,KitchenQual,27735.186441
4,BsmtExposure,27647.754558
5,SaleType_New,27593.351550
6,BsmtScore,27539.462582
7,LotArea,27491.871577
8,MasVnrArea,27453.207916
9,BedroomAbvGr,27416.685804


In [64]:
#This could be a promising method to use, with an R^2 of still 0.88 despite eliminating more than half the features
AmesDummiesForwardAIC = pd.concat([AmesDummiesMultiReduction[list(AmesDummiesForwardAICList['CreatedFeatures'])],
                                              AmesDummiesMultiReduction['SalePrice']], axis=1)

AmesDummiesOrdinalX = AmesDummiesForwardAIC.drop('SalePrice', axis=1)
AmesDummiesOrdinalY = AmesDummiesForwardAIC['SalePrice']

import statsmodels.api as sm
X = AmesDummiesOrdinalX
Y = AmesDummiesOrdinalY

X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     168.7
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:16:55   Log-Likelihood:                -13377.
No. Observations:                1166   AIC:                         2.690e+04
Df Residuals:                    1091   BIC:                         2.728e+04
Df Model:                          74                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.625e+

## Would our selected features differ drastically if we selected using Backwards selection?  If so, we should take any differences into account

In [65]:
#Define method to remove features based on those that most increase AIC:
def TrimFeatureListByAIC(df, dependent):
    AnyPositive = True
    df2 = df.copy()
    df2X = df2.drop(dependent, axis=1)
    df2Y = df2[dependent]
    StartingFeatureList = list(df2X.columns)
    RemovedFeatureList = []
    AICEvolutionList = []
    
    while AnyPositive == True:
        X2 = sm.add_constant(df2X)
        est = sm.OLS(df2Y, X2)
        est2 = est.fit()
        AICBase = est2.aic
        AICList = []
        
        for i in StartingFeatureList:
            tempDFX = df2X.drop(labels=i, axis=1)
            tempX2 = sm.add_constant(tempDFX)
            est = sm.OLS(df2Y, tempX2)
            est2 = est.fit()
            AICList.append(est2.aic)
            AICListN = np.array(AICList)
            
        if any(AICListN-AICBase < 0) == False:
            break
            
        else:
            index = AICList.index(min(AICList))
            RemovedValue = StartingFeatureList[index]
        RemovedFeatureList.append(RemovedValue)
        AICEvolutionList.append(AICList[index])
        df2X = df2X.drop(RemovedValue, axis=1)
        StartingFeatureList = list(df2X.columns)
    
    resultDF = pd.DataFrame({'RemovedFeatures': np.array(RemovedFeatureList), 'NewFScore': np.array(AICEvolutionList)})
        
    return resultDF

In [66]:
AmesDummiesBackwardAICList = TrimFeatureListByAIC(AmesDummiesMultiReduction, 'SalePrice')

In [67]:
AmesDummiesBackwardAICList

,RemovedFeatures,NewFScore
0,Neighborhood_Mitchel,27023.200584
1,Neighborhood_ClearCr,27021.200648
2,HouseStyle_1.5Unf,27019.201170
3,HeatingQC,27017.201729
4,Neighborhood_Gilbert,27015.203003
5,Exterior_HdBoard,27013.204418
6,LotShape_IR3,27011.206997
7,Exterior_BrkComm,27009.211461
8,MiscVal,27007.217822
9,MSSubClass_30,27005.231278


In [69]:
AmesDummiesForwardAICList.to_csv('AmesDummiesForwardAICListWith25.csv')
AmesDummiesBackwardAICList.to_csv('AmesDummiesBackwardAICRemovalListWith25.csv')